In [103]:
# Import core libraries
import pandas as pd
import numpy as np

# Load training dataset
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

# Combine train and test before feature engineering

# Add a column to identify source
train['source'] = 'train'
test['source'] = 'test'

# Combine train and test for consistent preprocessing
data = pd.concat([train, test], ignore_index=True)

In [104]:
# Step 1: Missing Value Summary (for combined data) 

# Calculate missing values in combined dataset
missing = data.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)

# Create a summary DataFrame
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing Ratio (%)': (missing / len(data)) * 100,
    'Data Type': data[missing.index].dtypes
})

# Optional: preview top missing features
missing_df.head(20)

,Missing Count,Missing Ratio (%),Data Type
PoolQC,2909,99.657417,object
MiscFeature,2814,96.402878,object
Alley,2721,93.216855,object
Fence,2348,80.438506,object
MasVnrType,1766,60.500171,object
SalePrice,1459,49.982871,float64
FireplaceQu,1420,48.646797,object
LotFrontage,486,16.649538,float64
GarageCond,159,5.447071,object
GarageYrBlt,159,5.447071,float64


In [105]:
# Step 2: Missing Value Handling
# version 2.0
# 1. For categorical features where missing means 'not present', fill with 'None'
#    This preserves the meaning without losing the column
# 2. For numerical features where missing means 0 (e.g. no area, no count), fill with 0
# 3. Fill LotFrontage with the median value for each Neighborhood
# 4. For all other categorical features, fill missing with the most frequent value (mode)
# 5. For remaining numerical features, fill missing with median (robust to outliers)

# Fill 'None' for categorical features where NA means 'no such feature'
none_fill_cols = [
    'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu',
    'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
    'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
    'MasVnrType'
]

for col in none_fill_cols:
    if col in data.columns:
        data[col] = data[col].fillna('None')

# Fill 0 for numerical features where NA means 'no value' (e.g., no garage or basement)
zero_fill_cols = [
    'GarageYrBlt', 'GarageArea', 'GarageCars',
    'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
    'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
    'MasVnrArea'
]

for col in zero_fill_cols:
    if col in data.columns:
        data[col] = data[col].fillna(0)

# Fill LotFrontage by the median value within each Neighborhood group
if 'LotFrontage' in data.columns and 'Neighborhood' in data.columns:
    data['LotFrontage'] = data.groupby('Neighborhood')['LotFrontage'].transform(
        lambda x: x.fillna(x.median())
    )

# For remaining categorical features, fill NA with mode (most frequent value)
for col in data.select_dtypes(include='object').columns:
    if data[col].isnull().sum() > 0:
        data[col] = data[col].fillna(data[col].mode()[0])

# For remaining numerical features, fill NA with median (robust to outliers)
for col in data.select_dtypes(include=['number']).columns:
    if data[col].isnull().sum() > 0:
        data[col] = data[col].fillna(data[col].median())


In [106]:
#Step 3: Handle skewed numeric features 

from scipy.stats import skew
import numpy as np

# Select numeric features only (excluding ID and target)
numeric_cols = data.select_dtypes(include=['number']).columns
numeric_cols = [col for col in numeric_cols if col not in ['Id', 'SalePrice']]

# Compute skewness for each numeric column
skewness = data[numeric_cols].apply(lambda x: skew(x.dropna()))

# Select features with high skewness (threshold > 0.75)
skewed_feats = skewness[abs(skewness) > 0.75].index.tolist()

# Remove MSSubClass (though numeric, it's categorical)
if 'MSSubClass' in skewed_feats:
    skewed_feats.remove('MSSubClass')

# Apply log1p transformation to reduce skewness
for feat in skewed_feats:
    data[feat] = np.log1p(data[feat])

In [107]:
# Step 4: Create new engineered features

# 1. Total square footage including basement
data['TotalSF'] = data['TotalBsmtSF'] + data['1stFlrSF'] + data['2ndFlrSF']

# 2. Total number of bathrooms (weighted: full=1, half=0.5)
data['TotalBath'] = (data['FullBath'] + 0.5 * data['HalfBath'] +
                     data['BsmtFullBath'] + 0.5 * data['BsmtHalfBath'])

# 3. Has a pool (binary flag)
data['HasPool'] = (data['PoolArea'] > 0).astype(int)

# 4. Was the house remodeled? (1 if yes, 0 if not)
data['IsRemodeled'] = (data['YearBuilt'] != data['YearRemodAdd']).astype(int)

# 5. Age of the house at sale
data['HouseAge'] = data['YrSold'] - data['YearBuilt']

# 6. Years since last remodel
data['RemodAge'] = data['YrSold'] - data['YearRemodAdd']


In [108]:
# Step 5: One-Hot Encoding for any remaining categorical columns
X = pd.get_dummies(X)
X_test = pd.get_dummies(X_test)

# Align columns to ensure same features in train and test
X, X_test = X.align(X_test, join='left', axis=1, fill_value=0)


In [109]:
# Step 6: One-Hot Encode all categorical variables
data = pd.get_dummies(data)

In [110]:
# Step 7: Split back into train and test (assumes original train was first 1460 rows)
train_rows = 1460
train = data.iloc[:train_rows, :].copy()
test = data.iloc[train_rows:, :].copy()

In [111]:
# Step 8: Separate features and target variable
X = train.drop(columns=['SalePrice'])
y = train['SalePrice']
X_test = test.drop(columns=['SalePrice']) if 'SalePrice' in test.columns else test

In [112]:
# Step 9: Save processed data for modeling
# Before saving y
y = np.log1p(y)
X.to_csv("X_train_cleaned.csv", index=False)
y.to_csv("y_train.csv", index=False)
X_test.to_csv("X_test_cleaned.csv", index=False)
